In [1]:
# Regression functions for Final Project 

In [2]:
import csv
import pandas as pd
import numpy as np
from linearmodels.panel import PanelOLS


In [3]:
df_path = 'cleaned_final_ver2.csv'

In [4]:
df =  pd.read_csv(df_path, encoding='ISO-8859-1') 

In [5]:
df.drop(index=1844, inplace=True)
df.drop(index=4230, inplace=True)

In [6]:
df = df[df["TotalRoomReceipts"] != 0]

In [7]:
df

,Unnamed: 0,publishedAtDate,title,address,city,state,totalScore,reviewsCount,reviewsDistribution,hotelStars,...,LocationAddress,LocationCity,LocationState,LocationZip,LocationCounty,UnitCapacity,ReportingQuarter,TotalRoomReceipts,TaxableReceipts,hotelid
0,0,202311,Sheraton Dallas Hotel,"400 Olive St, Dallas, TX 75201",Dallas,Texas,4.2,5918.0,"{'oneStar': 308, 'twoStar': 242, 'threeStar': ...",4-star hotel,...,400 OLIVE ST,DALLAS,TX,75201.0,57.0,1840.0,20231130.0,5078647.11,4407394.00,251.0
1,1,202310,Sheraton Dallas Hotel,"400 Olive St, Dallas, TX 75201",Dallas,Texas,4.2,5918.0,"{'oneStar': 308, 'twoStar': 242, 'threeStar': ...",4-star hotel,...,400 OLIVE ST,DALLAS,TX,75201.0,57.0,1840.0,20231031.0,6275946.25,6235523.00,251.0
2,2,202309,Sheraton Dallas Hotel,"400 Olive St, Dallas, TX 75201",Dallas,Texas,4.2,5918.0,"{'oneStar': 308, 'twoStar': 242, 'threeStar': ...",4-star hotel,...,400 OLIVE ST,DALLAS,TX,75201.0,57.0,1840.0,20230930.0,6084075.00,6077681.00,251.0
3,3,202308,Sheraton Dallas Hotel,"400 Olive St, Dallas, TX 75201",Dallas,Texas,4.2,5918.0,"{'oneStar': 308, 'twoStar': 242, 'threeStar': ...",4-star hotel,...,400 OLIVE ST,DALLAS,TX,75201.0,57.0,1840.0,20230831.0,4509562.10,4320492.00,251.0
4,4,202307,Sheraton Dallas Hotel,"400 Olive St, Dallas, TX 75201",Dallas,Texas,4.2,5918.0,"{'oneStar': 308, 'twoStar': 242, 'threeStar': ...",4-star hotel,...,400 OLIVE ST,DALLAS,TX,75201.0,57.0,1840.0,20230731.0,5285894.00,5280868.00,251.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4227,5055,202108,Kasa Greenville Dallas,"5065 Amesbury Dr, Dallas, TX 75206",Dallas,Texas,4.8,83.0,"{'oneStar': 1, 'twoStar': 2, 'threeStar': 1, '...",NaN,...,5065 AMESBURY DR,DALLAS,TX,75206.0,57.0,7.0,20210831.0,15503.79,11549.67,279.0
4228,5056,202109,Kasa Greenville Dallas,"5065 Amesbury Dr, Dallas, TX 75206",Dallas,Texas,4.8,83.0,"{'oneStar': 1, 'twoStar': 2, 'threeStar': 1, '...",NaN,...,5065 AMESBURY DR,DALLAS,TX,75206.0,57.0,7.0,20210930.0,10749.67,8668.99,279.0
4229,5057,202110,Kasa Greenville Dallas,"5065 Amesbury Dr, Dallas, TX 75206",Dallas,Texas,4.8,83.0,"{'oneStar': 1, 'twoStar': 2, 'threeStar': 1, '...",NaN,...,5065 AMESBURY DR,DALLAS,TX,75206.0,57.0,7.0,20211031.0,22021.57,17646.31,279.0
4231,5063,202204,Kasa Greenville Dallas,"5065 Amesbury Dr, Dallas, TX 75206",Dallas,Texas,4.8,83.0,"{'oneStar': 1, 'twoStar': 2, 'threeStar': 1, '...",NaN,...,5065 AMESBURY DR,DALLAS,TX,75206.0,57.0,7.0,20220430.0,32927.00,32927.00,279.0


In [8]:
df['revenue'] =  np.log(df['TotalRoomReceipts']+1)
rating = df['present_stars']
by_hotel = df['hotelid']

# Convert 'PublishedAtDate' to datetime format
df['publishedAtDate'] = pd.to_datetime(df['publishedAtDate'])

# Create entity and time indices
df = df.set_index(['hotelid', 'publishedAtDate'])

# Create Threshold for later regressions
df['T'] = (df['actual_stars'] < df['present_stars']).astype(int)
actual_rating = df['actual_stars']

In [9]:
# 1st regression ln_revenue = rating 

# Define and fit the model
model_1 = PanelOLS.from_formula('revenue ~ rating', data=df)
results_1 = model_1.fit()

# Print the title and then the regression results
print("Regression 1 Results")
print("================================================================================")
print(results_1)


Regression 1 Results
                          PanelOLS Estimation Summary                           
Dep. Variable:                revenue   R-squared:                        0.9874
Estimator:                   PanelOLS   R-squared (Between):              0.9771
No. Observations:                4124   R-squared (Within):              -0.9191
Date:                Fri, Mar 01 2024   R-squared (Overall):              0.9874
Time:                        12:01:05   Log-likelihood                   -7229.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   3.242e+05
Entities:                         145   P-value                           0.0000
Avg Obs:                       28.441   Distribution:                  F(1,4123)
Min Obs:                       1.0000                                           
Max Obs:                       70.000   F-statistic (robust):          3.242e+05
       

In [10]:
# 2nd regression ln_revenue = rating + hotelid + publishedatdate(control for fixed effects)
# Define the model
model_2 = PanelOLS.from_formula('revenue ~ rating + EntityEffects + TimeEffects', data=df)

# Fit the model
results_2 = model_2.fit()

# Print the summary of the regression results
print("Regression 2 Results")
print("================================================================================")
print(results_2)

Regression 2 Results
                          PanelOLS Estimation Summary                           
Dep. Variable:                revenue   R-squared:                        0.0059
Estimator:                   PanelOLS   R-squared (Between):              0.1430
No. Observations:                4124   R-squared (Within):              -0.0099
Date:                Fri, Mar 01 2024   R-squared (Overall):              0.1404
Time:                        12:01:05   Log-likelihood                   -1232.8
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      23.552
Entities:                         145   P-value                           0.0000
Avg Obs:                       28.441   Distribution:                  F(1,3944)
Min Obs:                       1.0000                                           
Max Obs:                       70.000   F-statistic (robust):             23.552
       

In [11]:
# 3rd regression
# Define the model
model_3 = PanelOLS.from_formula('revenue ~ T + actual_rating + EntityEffects + TimeEffects', data=df)

# Fit the model
results_3 = model_3.fit()

# Print the summary of the regression results
print("Regression 3 Results")
print("================================================================================")
print(results_3)

Regression 3 Results
                          PanelOLS Estimation Summary                           
Dep. Variable:                revenue   R-squared:                        0.0094
Estimator:                   PanelOLS   R-squared (Between):              0.1597
No. Observations:                4124   R-squared (Within):              -0.0083
Date:                Fri, Mar 01 2024   R-squared (Overall):              0.1568
Time:                        12:01:05   Log-likelihood                   -1225.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      18.650
Entities:                         145   P-value                           0.0000
Avg Obs:                       28.441   Distribution:                  F(2,3943)
Min Obs:                       1.0000                                           
Max Obs:                       70.000   F-statistic (robust):             18.650
       

In [12]:
# 4th regression
# Define the model
# rating:reviewLength is the interaction term
model_4 = PanelOLS.from_formula('revenue ~ T + rating + reviewLength + rating:reviewLength + EntityEffects + TimeEffects', data=df)

# Fit the model
results_4 = model_4.fit()

# Print the summary of the regression results
print("Regression 4 Results")
print("================================================================================")
print(results_4)

Regression 4 Results
                          PanelOLS Estimation Summary                           
Dep. Variable:                revenue   R-squared:                        0.0118
Estimator:                   PanelOLS   R-squared (Between):              0.1410
No. Observations:                4124   R-squared (Within):              -0.0022
Date:                Fri, Mar 01 2024   R-squared (Overall):              0.1383
Time:                        12:01:06   Log-likelihood                   -1220.7
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      11.717
Entities:                         145   P-value                           0.0000
Avg Obs:                       28.441   Distribution:                  F(4,3941)
Min Obs:                       1.0000                                           
Max Obs:                       70.000   F-statistic (robust):             11.717
       

In [13]:
# 5th regression
# Define the model
# rating:reviewContained is the interaction term
model_5 = PanelOLS.from_formula('revenue ~ T + rating + reviewContained + rating:reviewContained + EntityEffects + TimeEffects', data=df)

# Fit the model
results_5 = model_5.fit()

# Print the summary of the regression results
print("Regression 5 Results")
print("================================================================================")
print(results_5)

Regression 5 Results
                          PanelOLS Estimation Summary                           
Dep. Variable:                revenue   R-squared:                        0.0276
Estimator:                   PanelOLS   R-squared (Between):              0.1994
No. Observations:                4124   R-squared (Within):               0.0059
Date:                Fri, Mar 01 2024   R-squared (Overall):              0.1960
Time:                        12:01:06   Log-likelihood                   -1187.4
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.936
Entities:                         145   P-value                           0.0000
Avg Obs:                       28.441   Distribution:                  F(4,3941)
Min Obs:                       1.0000                                           
Max Obs:                       70.000   F-statistic (robust):             27.936
       